# Загрузка данных

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/train.csv').drop(columns=['index'])

In [3]:
df['class'] = pd.Categorical(df['class'])

In [4]:
df.head(3)

,text,class
0,Выплата на содержание родных детей,TAX
1,Оплата за Бритва Gillette Blue II Plus однораз...,NON_FOOD_GOODS
2,Паевой взнос в жилищно-строительный кооператив...,REALE_STATE


## Очистка датасета

In [5]:
!pip install natasha

  Obtaining dependency information for natasha from https://files.pythonhosted.org/packages/32/9c/bb9d33c13564bcc939bb727087ef51b16ed3b49cc3b8fdec07c87b02f1de/natasha-1.6.0-py3-none-any.whl.metadata
  Obtaining dependency information for pymorphy2 from https://files.pythonhosted.org/packages/07/57/b2ff2fae3376d4f3c697b9886b64a54b476e1a332c67eee9f88e7f1ae8c9/pymorphy2-0.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for razdel>=0.5.0 from https://files.pythonhosted.org/packages/15/2c/664223a3924aa6e70479f7d37220b3a658765b9cfe760b4af7ffdc50d38f/razdel-0.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for navec>=0.9.0 from https://files.pythonhosted.org/packages/bc/c1/771ec5565f0ce24874d7fd325b429f9caa80517a40d2e4ce5705120591f3/navec-0.10.0-py3-none-any.whl.metadata
  Obtaining dependency information for slovnet>=0.6.0 from https://files.pythonhosted.org/packages/7c/32/d5aff64e3d51ec4021674215680f16b7d2907860c6443b0d058579ac7d59/slovnet-0.6.0-py3-none-an

In [6]:
from natasha import Doc, Segmenter, MorphVocab
from natasha import NewsEmbedding, NewsMorphTagger
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [7]:

# Инициализация компонентов Natasha
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

# Список месяцев
months = [
    "январь", "февраль", "март", "апрель", "май", "июнь",
    "июль", "август", "сентябрь", "октябрь", "ноябрь", "декабрь"
]

# Стоп-слова
russian_stopwords = set(stopwords.words("russian"))
russian_stopwords.discard('за')  # Убираем из стоп-слов "за"
russian_stopwords.update(['кг', 'сумма', 'тч', 'мл', 'счёт'])  # Добавляем свои слова

# Функция очистки текста
def clean_text(text, russian_stopwords, segmenter, morph_vocab, morph_tagger, months):
    # Удаляем даты из текста
    date_pattern = r"(\d{0,4}[./-]?)"
    cleaned_text = re.sub(date_pattern, "", text)
    
    # Токенизация и удаление стоп-слов
    tokens = word_tokenize(cleaned_text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords]
    tokens = [token for token in tokens if token not in string.punctuation]
    cleaned_text = ' '.join(tokens)
    
    # Используем Natasha для морфологической обработки
    doc = Doc(cleaned_text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    
    # Лемматизация
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    
    # Убираем месяцы и короткие слова
    lemmas = [token.lemma for token in doc.tokens if token.lemma not in months and len(token.lemma) > 1]
    
    return ' '.join(lemmas)

In [9]:
df['clean_text'] = df['text'].apply(lambda x: clean_text(x,russian_stopwords, segmenter, morph_vocab, morph_tagger, months))
df['clean_text']

0                      выплата содержание родный ребенок
1      оплата за бритва gillette blue ii plus однораз...
2      паевой взнос жилищностроительный кооператив це...
3        за доставка жд транспорт одинцово бийск договор
4                          за юридический услуга договор
                             ...                        
395              оплата за восточный сладость халва счет
396    за транспортировка перевозка железный дорога б...
397                    оплата дог за гсоглсч счетфактура
398    оплата за масло оливковый global village class...
399       оплата за крем универсальный evo пантенол счет
Name: clean_text, Length: 400, dtype: object

# Наивный байесовский классификатор

In [10]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score, f1_score

In [11]:
vectorizer = TfidfVectorizer()

### На грязном тексте

In [12]:
X = vectorizer.fit_transform(df.text.tolist())
y = df['class']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

MultinomialNB()

In [15]:
y_pred = nb_model.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'F1-Score: {f1_score(y_test, y_pred, average="macro")}')

Accuracy: 0.8375
F1-Score: 0.8051641372524225


### На чистом тексте

In [16]:
X = vectorizer.fit_transform(df.clean_text.tolist())
y = df['class']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

MultinomialNB()

In [19]:
y_pred = nb_model.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'F1-Score: {f1_score(y_test, y_pred, average="macro")}')

Accuracy: 0.9125
F1-Score: 0.9255278565623394
